>动态规划（英语：Dynamic programming，简称DP）是一种在数学、管理科学、计算机科学、经济学和生物信息学中使用的，通过把原问题分解为相对简单的子问题的方式求解复杂问题的方法。

动态规划其实质上是通过开辟记录表，记录已求解过的结果，当再次需要求解的时候，可以直接到那个记录表中去查找，从而避免重复计算子问题来达到降低时间复杂度的效果。实际上是一个空间换时间的算法。动态规划，通常可以把指数级的复杂度降低到多项式级别。

动态规划很像分治法，都是通过分解成子问题并结合其结果来得到原问题的解。区别在于法的子问题不相交，但是动态规划的子问题是重叠的。
所以动态规划算法的有效性依赖于待求解问题本身具有的两个重要性质：最优子结构性质和子问题重叠性质。

动态规划在查找有很多重叠子问题的情况的最优解时有效。它将问题重新组合成子问题。为了避免多次解决这些子问题，它们的结果都逐渐被计算并被保存，从简单的问题直到整个问题都被解决。因此，动态规划保存递归时的结果，因而不会在解决同样的问题时花费时间。

动态规划只能应用于有最优子结构的问题。最优子结构的意思是局部最优解能决定全局最优解（对有些问题这个要求并不能完全满足，故有时需要引入一定的近似）。简单地说，问题能够分解成子问题来解决。

## 钢条切割问题
将长度为n的钢条切割成若干短钢条出售，切割成本不计，钢条长度与售价关系如下，求一种售价最大的切割法。
p={0:0,1:1,2:5,3:8,4:9,5:10,6:17,7:17,8:20,9:24,10:30}

## 常用的递归求解

In [72]:
p=[0,1,5,8,9,10,17,17,20,24,30]

def cut_rod(n,p):
    # 终止条件
    if n == 0 :
        return 0
    # 初始化
    q = -1
    
    for i in range(1, n+1):
        tmp = p[i] +cut_rod(n-i, p)
        
        # 记录最大值
        if tmp > q :
            q = tmp
    return q

%timeit x = cut_rod(8,p)
%timeit y = cut_rod(9,p)
print ("n=9的结果:",y)

1000 loops, best of 3: 165 µs per loop
1000 loops, best of 3: 308 µs per loop
n=9的结果: 25


但是这种方法对同样的规模的问题进行了多次重复的运算，即对2^(n-1)种切割都进行了计算，规模越小的问题重复次数越多，性能很差，时间性能为O(2^n).

## 动态规划
动态规划仔细安排求解，对每个子问题只求解一次，将结果保存，用额外的空间换取时间。有两种等价的实现方式：
###  带备忘的自顶向下
仍然按自然的递归形式编写过程，但过程中保存每个子问题的解，遇到子问题时先从存档中查找，若无记录再计算。

In [73]:
def memoized_cut_rod(n, p) :
    # 记录结果用的
    mem = [-1]*(n+1)
    
    def inner_aux (n,p):
        
        # 如果结果有了，就不用再次递归了
        if mem[n] >= 0:
            return mem[n]
        
        # 初始化
        if n==0:
            q = 0
        else :
            q = -1
            
        for i in range(1, n+1):
            tmp = p[i] +inner_aux(n-i, p)
            if tmp > q :
                q = tmp
        
        mem[n] = q
        return q
        
    return inner_aux(n, p)
%timeit t = memoized_cut_rod(8, p)
%timeit d = memoized_cut_rod(9, p)
print ("n=9的结果:", memoized_cut_rod(9, p))

10000 loops, best of 3: 23.3 µs per loop
10000 loops, best of 3: 27.9 µs per loop
n=9的结果: 25


### 自底向上
可以将子问题按规模排序，由小到进行求解，确保每次求解更小的子问题已有存档。

In [74]:
def bottom_up_cut_rod(n,p):
    r = [-1]*(n+1)
    r[0] = 0
    for i in range(1,n+1):
        q = -1
        for j in range(1,i+1):
            tmp = p[j] + r[i-j]
            if tmp >q :
                q = tmp
        r[j] = q
    return r[n]
%timeit e = bottom_up_cut_rod(8,p)
%timeit f = bottom_up_cut_rod(9,p)
print("n=9的结果",f)

100000 loops, best of 3: 14.3 µs per loop
100000 loops, best of 3: 17.1 µs per loop
n=9的结果 25


以上两种方法的渐近时间是一样的，为多项式级性能，只在特定的情形下有差异。由于bottom up的方法没有递归中频繁的函数调用开销，性能更好。